***NOTE: Live Monitoring Script runs before uWorc pipelines refresh the backend source (before 12:15PM UTC)***

In [10]:
import pandas as pd
import numpy as np
import warnings
import os
import ujson
import time
import sys

stdout = sys.stdout
import gspread
from googleapiclient.discovery import build

from oauth2client.service_account import ServiceAccountCredentials
import gspread_dataframe as gd
from queryrunner_client.muttley import MuttleyClient

from queryrunner_client import Client
from datetime import  date,datetime,timedelta
import datetime
import pytz
import calendar
import importlib

## Mailer
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import COMMASPACE
from email import encoders

reload(sys)

In [11]:
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1R2OOyDFBpJ9R9OxA_0-PcCmgi_oSjXyiJfOaaiRfpJg/edit?usp=sharing'
SPREADSHEET_ID = '1R2OOyDFBpJ9R9OxA_0-PcCmgi_oSjXyiJfOaaiRfpJg'

# query report id
reportid = 'wNRjvJ6zr'
df_freshness_reportid = '7lfaFmdDX'

email = 'rajansharma@uber.com'
x = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

scopes = ['https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scopes)

client = gspread.authorize(creds)

sh = client.open_by_url(spreadsheet_url)
qr = Client(user_email = email)

In [12]:
def pull_sheet_data(scopes,SPREADSHEET_ID,RANGE_NAME):
    
    """Pull data from G-sheets 

    Parameters:
    SCOPES : OAuth 2.0 scopes that you might need to request to access Google APIs, 
    depending on the level of access you need.
    
    SPREADSHEET_ID : The ID from the G-sheet URL
    
    RANGE_NAME : Specify the range of data user needs to pull
    
    Returns: Data from the spreadsheet 

   """
#     creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json',scopes)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=RANGE_NAME).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=RANGE_NAME).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data

### Pulling yesterdays run

In [13]:
# Pulling 'Run1' data
RANGE_NAME = 'Source FT per segment!A4:I'
data = pull_sheet_data(scopes,SPREADSHEET_ID,RANGE_NAME)
df = pd.DataFrame(data[1:], columns=data[0])

In [14]:
df

,ft_month,ft_week,sfdc_service_country,segment,lead_source_ib_or_ob,first_trips,Sub_Region,country_rename_placeholder,quarter_plus_year
0,2021-03-01 0:00:00,2021-03-22 0:00:00,France,Enterprise,outbound,31,FBS,France,Q1 - 2021
1,2020-02-01 0:00:00,2020-02-03 0:00:00,Spain,Strategic,outbound,4,SE,Spain,Q1 - 2020
2,2021-01-01 0:00:00,2021-01-11 0:00:00,Sweden,SMB,inbound,8,NCEE,Sweden,Q1 - 2021
3,2021-04-01 0:00:00,2021-04-12 0:00:00,Italy,Enterprise,outbound,10,SE,Italy,Q2 - 2021
4,2020-03-01 0:00:00,2020-02-24 0:00:00,France,SMB,inbound,10,FBS,France,Q1 - 2020
5,2019-02-01 0:00:00,2019-01-28 0:00:00,Belgium,SMB,inbound,4,FBS,Belgium,Q1 - 2019
6,2021-01-01 0:00:00,2021-01-25 0:00:00,Kenya,SMB,inbound,3,SSA,Kenya,Q1 - 2021
7,2020-02-01 0:00:00,2020-02-24 0:00:00,South Africa,Enterprise,outbound,7,SSA,South Africa,Q1 - 2020
8,2019-05-01 0:00:00,2019-04-29 0:00:00,France,Strategic,inbound,1,FBS,France,Q2 - 2019
9,2020-01-01 0:00:00,2020-01-13 0:00:00,France,SMB,inbound,55,FBS,France,Q1 - 2020


In [15]:
# The total rows from yesterdays run
yst_run = df.shape[0]

In [16]:
# params = {
#     'cw_timerange_start': '2020-06-01',
#     'enter_countries_in_single_quotes_separated_by_commas': ''''United Kingdom','Ireland''',
#     'interval': '28'
# } 

### Run the report 'today' and check for the row count

In [18]:
query_op = qr.execute_report(reportid,datacenter = 'dca1', timeout = 3600)
df_new = pd.DataFrame(query_op.load_data(), columns = query_op.columns)

In [20]:
latest_run = df_new.shape[0]

### Aggregated values check

In [10]:
df.ft_month = pd.to_datetime(df.ft_month)
df.ft_week = pd.to_datetime(df.ft_week)

In [11]:
new_df = pd.merge(df[['ft_month','ft_week','sfdc_service_country','segment','lead_source_ib_or_ob','first_trips']], 
                  df_new[['ft_month','ft_week','sfdc_service_country','segment','lead_source_ib_or_ob','first_trips']],  
                  how='inner', 
                  left_on=['ft_month','ft_week','sfdc_service_country','segment','lead_source_ib_or_ob'],
                  right_on = ['ft_month','ft_week','sfdc_service_country','segment','lead_source_ib_or_ob'])

In [12]:
# aggvalues_check = new_df[(new_df.first_trips_x != new_df.first_trips_y)]

In [13]:
new_df.dtypes

ft_month                datetime64[ns]
ft_week                 datetime64[ns]
sfdc_service_country            object
segment                         object
lead_source_ib_or_ob            object
first_trips_x                   object
first_trips_y                    int64
dtype: object

In [14]:
new_df.first_trips_x = new_df.first_trips_x.astype('int64')
new_df.first_trips_y = new_df.first_trips_y.astype('int64')

In [15]:
# new_df[(new_df.first_trips_x != new_df.first_trips_y)]

In [16]:
new_df['delta'] = new_df['first_trips_x'] - new_df['first_trips_y']

In [17]:
new_df[new_df.delta > 2]

,ft_month,ft_week,sfdc_service_country,segment,lead_source_ib_or_ob,first_trips_x,first_trips_y,delta
197,2021-01-01,2021-01-25,United Kingdom,Enterprise,outbound,191,188,3
560,2021-02-01,2021-02-15,France,Enterprise,outbound,71,68,3
585,2020-11-01,2020-11-16,United Kingdom,Enterprise,outbound,84,80,4
1175,2020-11-01,2020-11-09,United Kingdom,Enterprise,outbound,149,145,4
2179,2019-03-01,2019-03-25,Ireland,Strategic,outbound,7,4,3
2417,2021-02-01,2021-02-22,South Africa,SMB,inbound,28,25,3
3012,2020-07-01,2020-06-29,United Kingdom,Enterprise,outbound,173,170,3
3248,2021-02-01,2021-02-22,United Kingdom,Enterprise,outbound,69,66,3
4040,2020-10-01,2020-10-19,United Kingdom,Enterprise,outbound,96,91,5
4551,2021-02-01,2021-02-01,United Kingdom,Enterprise,outbound,177,174,3


### DB Freshness check

In [81]:
query_op = qr.execute_report(df_freshness_reportid,datacenter = 'dca1', timeout = 3600)
db_freshness = pd.DataFrame(query_op.load_data(), columns = query_op.columns)

In [82]:
datetime_UTC = datetime.datetime.now(pytz.utc)
curr_date = datetime_UTC.strftime("%Y-%m-%d")
current_time = datetime_UTC.strftime("%H:%M:%S")

compare_time = datetime_UTC - pd.to_timedelta(2, unit='d')
compare_date = compare_time.strftime("%Y-%m-%d")

In [83]:
# db_freshness['current_date'] = curr_date

In [84]:
curr_date

'2021-02-25'

In [85]:
db_freshness

,subquery,latest_date
0,sfdc_onboarding,2021-02-25 00:01:39
1,salesforce_physical_locations,2021-02-24 23:38:56
2,salesforce_eats_account,2021-02-24 20:35:25
3,fact_eats_order_rank,2021-02-25 06:16:48
4,rawdata_user,2021-02-25 00:00:00
5,sfdc_opp_raw_table,2021-02-24 00:00:00


In [86]:
dbf_df = db_freshness[db_freshness.latest_date < compare_date]

In [87]:
dbf_df.shape[0]

0

In [94]:
# (yst_run > latest_run)
(yst_run > latest_run) and (aggvalues_check.shape[0] > 0) and (dbf_df.shape[0] > 0)
# or (aggvalues_check.shape[0] > 0)

False

## Send Mail

In [ ]:
SUBJECT = 'Data Refresh Failed - FT per Segment'
MY_EMAIL = 'rajansharma@uber.com,'
MY_PASSWORD = 'password'
# TO_EMAIL = 'rajansharma@uber.com,matt.mueller@uber.com,federico.marzotto@uber.com,emea-eats-sops-analytics-group@uber.com'
TO_EMAIL = 'rajansharma@uber.com'
SMTP_SERVER = '127.0.0.1'
SMTP_SERVER = '127.0.0.1'
rcpt = TO_EMAIL.split(",")

In [ ]:
if (yst_run > latest_run) and (aggvalues_check.shape[0] == 0) and (dbf_df.shape[0] == 0):

    msg = MIMEMultipart()
    msg['From'] = MY_EMAIL
    msg['To'] = TO_EMAIL
    msg['Subject'] = SUBJECT

    html = """\
           <html>
           <body>
           <p>Hi Team,<br>
           <br>
            The data refresh has failed on <b>"""+str(curr_date)+""", """+str(current_time)+ """.</b><br>
            <ul><li>Last successful run output : <b>"""+str(yst_run)+"""+ rows</b></li>
            <li>Today's run output : <b>"""+str(latest_run)+""" rows</b></li></ul>
            <br>

            Regards,<br>
            <b>EMEA Sales Ops Team<b>
            </p>
            </body>
            </html>
                   """

    # Turn these into plain/html MIMEText objects
    part1 = MIMEText(html, "html")
    msg.attach(part1)

    smtpObj = smtplib.SMTP(SMTP_SERVER)
    smtpObj.sendmail(MY_EMAIL, rcpt, msg.as_string())
    smtpObj.quit()
    
elif (yst_run > latest_run) and (aggvalues_check.shape[0] > 0) and (dbf_df.shape[0] == 0):


    msg = MIMEMultipart()
    msg['From'] = MY_EMAIL
    msg['To'] = TO_EMAIL
    msg['Subject'] = SUBJECT

    html = """\
           <html>
           <body>
           <p>Hi Team,<br>
           <br>
            The data refresh has failed on <b>"""+str(curr_date)+""", """+str(current_time)+ """.</b><br>
            <ul><li>Last successful run output : <b>"""+str(yst_run)+"""+ rows</b></li>
            <li>Today's run output : <b>"""+str(latest_run)+""" rows</b></li></ul>
            <br>
            The aggregated values for certain months do not match with the previous runs: 
            <br><br>
            {0}
            <br>
            PFB, the database freshness check has failed for the below sub-queries/tables :
            <br><br>
            {1}
            <br>

            Regards,<br>
            <b>EMEA Sales Ops Team<b>
            </p>
            </body>
            </html>
                   """.format(aggvalues_check.to_html())

    # Turn these into plain/html MIMEText objects
    part1 = MIMEText(html, "html")
    msg.attach(part1)

    smtpObj = smtplib.SMTP(SMTP_SERVER)
    smtpObj.sendmail(MY_EMAIL, rcpt, msg.as_string())
    smtpObj.quit()
    
elif (yst_run > latest_run) and (aggvalues_check.shape[0] > 0) and (dbf_df.shape[0] > 0):

    msg = MIMEMultipart()
    msg['From'] = MY_EMAIL
    msg['To'] = TO_EMAIL
    msg['Subject'] = SUBJECT

    html = """\
           <html>
           <body>
           <p>Hi Team,<br>
           <br>
            The data refresh has failed on <b>"""+str(curr_date)+""", """+str(current_time)+ """.</b><br>
            <ul><li>Last successful run output : <b>"""+str(yst_run)+"""+ rows</b></li>
            <li>Today's run output : <b>"""+str(latest_run)+""" rows</b></li></ul>
            <br>
            The aggregated values for certain months do not match with the previous runs: 
            <br><br>
            {0}
            <br>
            PFB, the database freshness check has failed for the below sub-queries/tables :
            <br><br>
            {1}
            <br>

            Regards,<br>
            <b>EMEA Sales Ops Team<b>
            </p>
            </body>
            </html>
                   """.format(aggvalues_check.to_html(),dbf_df.to_html())

    # Turn these into plain/html MIMEText objects
    part1 = MIMEText(html, "html")
    msg.attach(part1)

    smtpObj = smtplib.SMTP(SMTP_SERVER)
    smtpObj.sendmail(MY_EMAIL, rcpt, msg.as_string())
    smtpObj.quit()
    
elif (yst_run < latest_run) and (aggvalues_check.shape[0] > 0) and (dbf_df.shape[0] == 0):

    msg = MIMEMultipart()
    msg['From'] = MY_EMAIL
    msg['To'] = TO_EMAIL
    msg['Subject'] = SUBJECT

    html = """\
           <html>
           <body>
           <p>Hi Team,<br>
           <br>
            The data refresh has failed on <b>"""+str(curr_date)+""", """+str(current_time)+ """.</b><br>
            The aggregated values for certain months do not match with the previous runs: 
            <br><br>
            {0}
            <br>

            Regards,<br>
            <b>EMEA Sales Ops Team<b>
            </p>
            </body>
            </html>
                   """.format(aggvalues_check.to_html())

    # Turn these into plain/html MIMEText objects
    part1 = MIMEText(html, "html")
    msg.attach(part1)

    smtpObj = smtplib.SMTP(SMTP_SERVER)
    smtpObj.sendmail(MY_EMAIL, rcpt, msg.as_string())
    smtpObj.quit()

In [123]:
# if (yst_run > latest_run) or (aggvalues_check.shape[0] > 0) or (dbf_df.shape[0] > 0):
#     SUBJECT = 'Data Refresh Failed - FT per Segment'
#     MY_EMAIL = 'rajansharma@uber.com,'
#     MY_PASSWORD = 'password'
#     TO_EMAIL = 'rajansharma@uber.com,matt.mueller@uber.com,federico.marzotto@uber.com,emea-eats-sops-analytics-group@uber.com'
#     SMTP_SERVER = '127.0.0.1'
#     SMTP_SERVER = '127.0.0.1'
#     rcpt = TO_EMAIL.split(",")

#     msg = MIMEMultipart()
#     msg['From'] = MY_EMAIL
#     msg['To'] = TO_EMAIL
#     msg['Subject'] = SUBJECT

#     html = """\
#            <html>
#            <body>
#            <p>Hi Team,<br>
#            <br>
#             The data refresh has failed on <b>"""+str(curr_date)+""", """+str(current_time)+ """.</b><br>
#             <ul><li>Last successful run output : <b>"""+str(yst_run)+"""+ rows</b></li>
#             <li>Today's run output : <b>"""+str(latest_run)+""" rows</b></li></ul>
#             <br>
#             The aggregated values for certain months do not match with the previous runs: 
#             <br><br>
#             {0}
#             <br>
#             PFB, the database freshness check has failed for the below sub-queries/tables :
#             <br><br>
#             {1}
#             <br>
#             The data refresh has failed. We are troubleshooting the issue. We'll notify you once the problem is resolved.<br>
#             <br>


#             Regards,<br>
#             <b>EMEA Sales Ops Team<b>
#             </p>
#             </body>
#             </html>
#                    """.format(aggvalues_check.to_html(),dbf_df.to_html())

#     # Turn these into plain/html MIMEText objects
#     part1 = MIMEText(html, "html")
#     msg.attach(part1)

#     smtpObj = smtplib.SMTP(SMTP_SERVER)
#     smtpObj.sendmail(MY_EMAIL, rcpt, msg.as_string())
#     smtpObj.quit()